# finder
best model & tokenizer
load test dataset

(1번) masking 0th

(2번) masking 0th + result for all test datasets

(3-1번)

(3-1번 결과)

(3-2번)

(3-2번 결과)

In [1]:
import torch

# best model & tokenizer

In [19]:
from transformers import (AutoConfig, AutoModelForMaskedLM)

import bootstrap

from ingt_tokenizer import IngtTokenizer


VOCAB_CONFIG = "ingr_title_tag"
PROJECT_PATH= "/media/dc/Data2/recipebuild/mlm"
CONFIG_PATH = f"{PROJECT_PATH}/config.json"
ingt_config = bootstrap.IngTConfig(vocab=VOCAB_CONFIG, path=CONFIG_PATH)

ingt_tokenizer = IngtTokenizer(ingt_config)
ingt_tokenizer.load()
tokenizer = ingt_tokenizer.tokenizer

model_name_or_ckpt_path = f'{PROJECT_PATH}/checkpoints/v3-ing-title-tag-tagtokenizer_2023-09-05-11-54/checkpoint-125000'

config = AutoConfig.from_pretrained(            model_name_or_ckpt_path)

# _device = torch.device('cuda')
model = AutoModelForMaskedLM.from_pretrained(
    model_name_or_ckpt_path,
    config=config
)
# model.to(_device)

# load test dataset

In [3]:
# load test dataset
test_dataset_path = f'{PROJECT_PATH}/tester/recipebowl_test_data_with_tag_and_title.txt'

f = open(test_dataset_path, 'r')

lines = f.readlines()

cnt = 0
for line in lines:
    print(line)
    cnt += 1
    if cnt > 0 : break

Fontina_cheese fresh_basil_leaves elbow_macaroni garlic_cloves pine_nuts parmesan_cheese eggs butter[SEP]60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go[SEP]Pesto Shrimp Mac and Cheese



In [4]:
lines[0]

'Fontina_cheese fresh_basil_leaves elbow_macaroni garlic_cloves pine_nuts parmesan_cheese eggs butter[SEP]60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go[SEP]Pesto Shrimp Mac and Cheese\n'

In [5]:
def make_input(sentence, masking_position):
    list_of_input = sentence.split(' ')
    answer_sentence = sentence
    list_of_input[masking_position] = '[MASK]'
    input_sentence = ' '.join(list_of_input)
    return input_sentence, answer_sentence

In [6]:
input_sample = make_input(lines[0], 0)[0]

In [7]:
input_sample

'[MASK] fresh_basil_leaves elbow_macaroni garlic_cloves pine_nuts parmesan_cheese eggs butter[SEP]60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go[SEP]Pesto Shrimp Mac and Cheese\n'

# (1번) masking 0th

In [48]:
inputs = tokenizer(input_sample, return_tensors='pt')
outputs = model(**inputs)
print(torch.argmax(outputs.logits, dim=2).reshape(-1))
pred = tokenizer.convert_ids_to_tokens(torch.argmax(outputs.logits, dim=2).reshape(-1))

# top5
print(pred[0:5])

tensor([3982, 2275, 2770,  435, 2127,  722,  263,  284,  206, 1062,  235,  291,
         512,  368,  690,  940,  896,  490,  571,  524, 2339, 1026,  358,  703,
        1331, 2359,  896, 2368,  480,  490])
['asiago_cheese', 'fresh_basil_leaves', 'elbow_macaroni', 'garlic_cloves', 'pine_nuts']


In [51]:
outputs.logits.shape

torch.Size([1, 30, 199492])

In [49]:
tokenizer.convert_ids_to_tokens([0,1,2])

['[UNK]', '[PAD]', '[CLS]']

In [53]:
def get_res(ingredients, tags, title, masking_position, start_res=0, num_res=10):
    sentence = ' [SEP] '.join([' '.join(ingredients), ' '.join(tags), title ])
    # print(sentence)
    input = make_input(sentence, masking_position)[0]
    # print(input)
    inputs = tokenizer(input, return_tensors='pt')
    outputs = model(**inputs)
    # print(outputs.logits[0][masking_position].shape)
    topk_idxs = torch.topk(outputs.logits[0][masking_position], num_res)
    # print(topk_idxs)
    pred = tokenizer.convert_ids_to_tokens(topk_idxs.indices)

    # top5
    return {
        'sentence': sentence,
        'input': input,
        'res': pred[start_res:num_res]
    }
    


In [74]:
def exp1(current_input):
    print('=================================================')
    if 'masking_position' in current_input:
        del current_input['masking_position']
    print(current_input)
    for i in range(len(current_input['ingredients'])):
        print(f">> maksing position: {i} / {current_input['ingredients'][i]} removed ")
        current_input['masking_position'] = i
        print(get_res(**current_input)['res'])
        # print(get_res(['bazil', 'kimchi', 'rice'], [], 'Randokm fried rice', 2)['res'])

current_input = {
    'ingredients': ['pork', 'rice'],
    'tags': [], 
    'title': '',
    # 'masking_position': 0
} 
exp1(current_input)

current_input['tags'] = ['Indian']
exp1(current_input)

current_input['tags'] = ['Korean']
exp1(current_input)

current_input['tags'] = ['Italian']
exp1(current_input)

current_input['title'] = 'Fried rice' 
current_input['tags'] = [] 
exp1(current_input)

current_input['tags'] = ['Indian']
exp1(current_input)

current_input['tags'] = ['Korean']
exp1(current_input)

current_input['tags'] = ['Italian']
exp1(current_input)




{'ingredients': ['pork', 'rice'], 'tags': [], 'title': ''}
>> maksing position: 0 / pork removed 
['peas', 'soup', 'french_onion_soup', 'broth', 'chicken_broth', 'onion_soup', 'chicken_stock_cubes', 'sausage', 'stock', 'cassava']
>> maksing position: 1 / rice removed 
['soy_sauce', 'meat', 'pork', 'paprika', 'barbecue_sauce', 'pork_chops', 'cream_of_mushroom_soup', 'potatoes', 'pork-loins', 'pork_roast']
{'ingredients': ['pork', 'rice'], 'tags': ['Indian'], 'title': ''}
>> maksing position: 0 / pork removed 
['ghee', 'turmeric_powder', 'chutney', 'spices', 'channa_dal', 'yogurt', 'chat_masala', 'cardamom_powder', 'cassava', 'coconut']
>> maksing position: 1 / rice removed 
['turmeric_powder', 'cooking_oil', 'red_chili_powder', 'oil', 'green_chilies', 'garam_masala', 'mustard_seeds', 'green_chili_peppers', 'potatoes', 'fennel_seeds']
{'ingredients': ['pork', 'rice'], 'tags': ['Korean'], 'title': ''}
>> maksing position: 0 / pork removed 
['beef', 'kimchi', 'hot_pepper_paste', 'kimchee',

# (2번) masking 0th + result for all test datasets

In [9]:
# (2번) masking 0th + result for all test datasets

import pandas as pd 

file_path2 = f'{PROJECT_PATH}/tester/test_output/ing_title_tag_test_125K_with_tagtokenizer_2023_0910-0.38259474338014265-0.2867754088534977-0.510488911721894-0.5977395324942204-0.5977395324942204.tsv'

result2 = pd.read_csv(file_path2, sep='\t')

In [10]:
result2.head(2)

,original,q,answer,ori_sp,ori_len,q_tokenized,predicted #0,predicted #1,predicted #2,predicted #3,predicted #4,predicted #5,predicted #6,predicted #7,predicted #8,predicted #9,errata_1,errata_5,errata_10,errata_20
0,Fontina_cheese fresh_basil_leaves elbow_macaro...,[MASK] fresh_basil_leaves elbow_macaroni garli...,Fontina_cheese,"['Fontina_cheese', 'fresh_basil_leaves', 'elbo...",26,"{'input_ids': tensor([[ 4, 2275, 2770, 435,...",asiago_cheese,fontina_cheese,brie_cheese,baby_shrimp,italian_cheese_blend,cooked_shrimp,alfredo_sauce,romano_cheese,pesto_sauce,prawns,0,0,0,0
1,yellow_bell_peppers low_sodium_soy_sauce brocc...,[MASK] low_sodium_soy_sauce broccoli_florets r...,yellow_bell_peppers,"['yellow_bell_peppers', 'low_sodium_soy_sauce'...",19,"{'input_ids': tensor([[ 4, 2965, 2588, 1309,...",dark_sesame_oil,snow_peas,minced_ginger,bean_sprouts,rice_noodles,yellow_bell_peppers,baby_corn,grapeseed_oil,sliced_water_chestnuts,linguine,0,0,1,1


In [15]:
model.device

device(type='cpu')

# (3-1번)

In [20]:
top_k = 10
_MASK_NUM = 4

my_answer_sentence = []
my_answer_ing_list = []
my_model_input = []
my_pred_topk_list = []
my_answer_ing_count = []
my_index_per_row = [] 
my_score = []
# with open(test_dataset_path, 'r') as f2:


score_list = []
for idx, line in enumerate(lines[0:5]):
    ##
    model_output_list = []

    line = line.strip()
    
    if len(line) == 0:
        continue

    line_split = line.split('[SEP]')
    ing_list = line_split[0]
    tag_list = line_split[1]
    title_list = line_split[2]

    answer_ing_list = ing_list.split(' ')

    tag_list_count = len(tag_list.split(' '))
    title_list_count = len(title_list.split(' '))
    index_for_logits = tag_list_count + title_list_count

    tag_list.split(' ')

    answer_sentence = tag_list + ' [SEP] ' + title_list + ' [SEP] ' + ing_list

    answer_ing_count = len(ing_list.split(' '))

    _ing_accumulate_list = []

    for i, _ing in enumerate(ing_list.split(' ')):
        
        cnt = 0

        _ing_accumulate_list.append(_ing)
        answer_sen = tag_list + ' [SEP] ' + title_list + ' [SEP] ' + ' '.join(_ing_accumulate_list)

        _ing_accumulate_list[i] = '[MASK]'
        answer_ing = _ing
        model_input = tag_list + ' [SEP] ' + title_list + ' [SEP] ' + ' '.join(_ing_accumulate_list)

        model_output = []

        # model.to(_device)

        _topk_num = top_k
        # _pred_topk_list = [[] for z in range(_topk_num)]
        _pred_topk_list = []

        # model predict
        _tokenized = tokenizer(str(model_input), return_tensors="pt")

        # _t = {k:v.to(_device) for k, v in _tokenized.items()}
        _t = {k:v for k, v in _tokenized.items()}

        _pred = model(**_t)

        _MASK_IDX = torch.where(_t['input_ids'][0] == _MASK_NUM)[0][0].detach().cpu()
        # _logit = _pred.logits[0][index_for_logits+2+i].detach().cpu()
        _logit = _pred.logits[0][_MASK_IDX].detach().cpu()
        # import IPython; IPython.embed(color="Linux"); exit(1)
        del _pred.logits
        del _pred
        del _t
        del _tokenized
        torch.cuda.empty_cache()
        model_output.append(_logit)

        _res = tokenizer.convert_ids_to_tokens(torch.topk(_logit, _topk_num).indices)

        for _i, _token in enumerate(_res):
            # _pred_topk_list[_i].append(_token)
            _pred_topk_list.append(_token)

        if answer_ing in _pred_topk_list:
            cnt += 1

        _ing_accumulate_list[i] = _ing
                        
        model_output_list.append(_pred_topk_list)
        my_model_input.append(model_input)
        my_index_per_row.append(f"{i+1}개")

        score = cnt/answer_ing_count

        score_list.append(score)

        # if answer_ing == 'water':
        #     import IPython; IPython.embed(color="Linux"); exit(1)

    my_answer_sentence.append(answer_sentence)
    my_pred_topk_list.append(model_output_list)
    my_answer_ing_count.append(f"총 {answer_ing_count}개")
    my_score = score_list
    my_answer_ing_list.append(answer_ing_list)


        # cnt_list.append(score)
        # print(Fraction(cnt/answer_ing_count))


df = pd.DataFrame()
df['answer_sentence'] = my_answer_sentence
df['answer_ing'] = my_answer_ing_list 
df['model_output_list'] = my_pred_topk_list
df['ing_count'] = my_answer_ing_count
# df['score'] = my_score

df = df.explode(['answer_ing','model_output_list'], ignore_index=True)
df['score'] = my_score
df['model_input'] = my_model_input
df['input_ing_count'] = my_index_per_row

# df = df[['answer_sentence', 'model_input', 'answer_ing', 'model_output_list', 'input_ing_count', 'ing_count', 'score']]

df = df[['answer_sentence', 'model_input', 'answer_ing', 'model_output_list', 'input_ing_count', 'ing_count', 'score']]
    

In [21]:
df.head()

,answer_sentence,model_input,answer_ing,model_output_list,input_ing_count,ing_count,score
0,60-minutes-or-less occasion main-dish eggs-dai...,60-minutes-or-less occasion main-dish eggs-dai...,Fontina_cheese,"[cheesy, cheese, skillet, easy, velveeta, maca...",1개,총 8개,0.0
1,60-minutes-or-less occasion main-dish eggs-dai...,60-minutes-or-less occasion main-dish eggs-dai...,fresh_basil_leaves,"[macaroni, bake, casserole, pasta, skillet, sa...",2개,총 8개,0.0
2,60-minutes-or-less occasion main-dish eggs-dai...,60-minutes-or-less occasion main-dish eggs-dai...,elbow_macaroni,"[and, pasta, &, with, macaroni, bake, casserol...",3개,총 8개,0.0
3,60-minutes-or-less occasion main-dish eggs-dai...,60-minutes-or-less occasion main-dish eggs-dai...,garlic_cloves,"[pasta, alfredo_sauce, penne, condensed_cream_...",4개,총 8개,0.0
4,60-minutes-or-less occasion main-dish eggs-dai...,60-minutes-or-less occasion main-dish eggs-dai...,pine_nuts,"[alfredo_sauce, pesto_sauce, pasta-rice-and-gr...",5개,총 8개,0.0


In [22]:
cnt = 0
for i, v in enumerate(zip(df['answer_sentence'], df['model_input'], df['answer_ing'], df['model_output_list'])):
    print(v[0])
    print(v[1])
    print(v[2])
    print(v[3])

    cnt += 1
    print('')

    if cnt > 3: break

60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go [SEP] Pesto Shrimp Mac and Cheese [SEP] Fontina_cheese fresh_basil_leaves elbow_macaroni garlic_cloves pine_nuts parmesan_cheese eggs butter
60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go [SEP] Pesto Shrimp Mac and Cheese [SEP] [MASK]
Fontina_cheese
['cheesy', 'cheese', 'skillet', 'easy', 'velveeta', 'macaroni', 'pasta', 'gourmet', 'alfredo_sauce', 'lite']

60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go [SEP] Pesto Shrimp Mac and Cheese [SEP] Fontina_cheese fresh_basil_leaves elbow_macaroni garlic_cloves pine_nuts parmesan_cheese eggs butter
60-minutes-or-less occasion main-dish eggs-dairy pasta seafo

# (3-1번 결과)

In [23]:
file_path31 = f'{PROJECT_PATH}/tester/test_output/10-next_token_predict_2023_1027_all2.tsv' # top_k = 10

tt1 = pd.read_csv(file_path31, sep='\t')

In [24]:
tt1.head(2)

,IDX,SUB_IDX,ORIGINAL,MODEL_INPUT,ANSWER,PRED,SCORE,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9,pred10
0,1,1,Fontina_cheese fresh_basil_leaves elbow_macaro...,[MASK] [SEP] 60-minutes-or-less occasion main-...,Fontina_cheese,"['alfredo_sauce', 'pesto_sauce', 'parmesan_che...",0,alfredo_sauce,pesto_sauce,parmesan_cheese,milk,ranch_dressing,shrimp,mayonnaise,nonfat_milk,mozzarella_cheese,heavy_cream
1,1,2,Fontina_cheese fresh_basil_leaves elbow_macaro...,Fontina_cheese [MASK] [SEP] 60-minutes-or-less...,fresh_basil_leaves,"['alfredo_sauce', 'olive_oil', 'parmesan_chees...",0,alfredo_sauce,olive_oil,parmesan_cheese,butter,milk,shrimp,garlic_cloves,unsalted_butter,elbow_macaroni,heavy_cream


In [25]:
lines[0:5]

['Fontina_cheese fresh_basil_leaves elbow_macaroni garlic_cloves pine_nuts parmesan_cheese eggs butter[SEP]60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go[SEP]Pesto Shrimp Mac and Cheese\n',
 'yellow_bell_peppers low_sodium_soy_sauce broccoli_florets red_onions water_chestnuts rice_vinegar snow_peas shrimp mushrooms garlic[SEP]weeknight 60-minutes-or-less cuisine occasion seafood asian shrimp shellfish[SEP]Shrimp Stir Fry\n',
 'pork_chops cooking_oil apples brown_sugar cinnamon butter[SEP]low-protein very-low-carbs casseroles main-dish pork oven easy low-sodium low-cholesterol low-saturated-fat low-calorie low-carb meat pork-chops number-of-servings 4-hours-or-less[SEP]Pork Chops and Apples\n',
 'havarti_cheese boneless_chicken_breasts roma_tomatoes romaine_lettuce black_olives basil_leaves croutons parmesan_cheese carrots mushrooms[SEP]60-minutes-or-less occasion main-dish

# (3-2번)

In [27]:
top_k = 1

kk = open(f'{PROJECT_PATH}/tester/ing_list.txt')
lines2 = kk.readlines()

ing_list = []
for line2 in lines2:
    ing_list.append(line2.replace("\n",""))

IDX = [] ##
SUB_IDX = [] ## 
SEN_ORIGINAL = [] ## 
MODEL_INPUT = [] ##
ANSWER = [] ##
PRED = [] ##
SCORE = [] ##

ANSWER_V2 = []; PRED_V2 = []; 
IDX_V2 = []; SEN_ORIGINAL_V2 = []; TITLE = []; MODEL_INPUT_V2 = []

cnt1 = 0
for idx, line in (enumerate(lines[0:5])):
    line = line.strip()

    if len(line) == 0:
        continue

    split_line = line.split('[SEP]')
    list_ing = split_line[0]
    list_tag = split_line[1]
    list_title = split_line[2]

    answer = list_ing.split(' ')
    
    cnt1 += 1

    # additionally
    IDX_V2.append(cnt1)
    SEN_ORIGINAL_V2.append(line)
    TITLE.append(list_title)


    temp = []; answer = []; list_model_input = []; cnt2 = 0
    score = []; answer_v2 = []; pred_v2 = []
    for i, _ing in enumerate(list_ing.split(' ')):
        IDX.append(cnt1)
        SEN_ORIGINAL.append(line)
        ANSWER.append(_ing)
            
        # additionally
        answer_v2.append(_ing)

        answer_cnt = 0

        temp.append(_ing)
        # masking_sen_answer = list_tag + ' [SEP] ' + list_title + ' [SEP] ' + ' '.join(temp)
        masking_sen_answer = ' '.join(temp) + ' [SEP] ' + list_tag + ' [SEP] ' + list_title

        temp[i] = '[MASK]'

        # model_input = list_tag + ' [SEP] ' + list_title + ' [SEP] ' + ' '.join(temp)
        model_input = ' '.join(temp) + ' [SEP] ' + list_tag + ' [SEP] ' + list_title
        MODEL_INPUT.append(model_input)
        cnt2 += 1
        SUB_IDX.append(cnt2)

        model_output = []

        # _device = torch.device('cuda:1')
        # model.to(_device)

        _topk_num = top_k
        pred = []

        _tokenized = tokenizer(str(model_input), return_tensors="pt")
        # _t = {k: v.to(_device) for k, v in _tokenized.items()}
        _t = {k: v for k, v in _tokenized.items()}

        _pred = model(**_t)

        _MASK_IDX = torch.where(_t['input_ids'][0]==_MASK_NUM)[0][0].detach().cpu()
        _logit = _pred.logits[0][_MASK_IDX].detach().cpu()

        del _pred.logits
        del _pred
        del _t
        del _tokenized
        torch.cuda.empty_cache()
        model_output.append(_logit)
        
        _res = tokenizer.convert_ids_to_tokens(torch.topk(_logit, _topk_num).indices)

        ###
        start = top_k; count = 0
        while len([1 for s in _res if s in ing_list]) != top_k: 

            for j, v in enumerate(_res):
                if v not in ing_list:
                    _res.remove(v)

            end = start + (top_k-len(_res))

            tmp = tokenizer.convert_ids_to_tokens(torch.sort(_logit)[1][::1].flip(dims=(0,))[start:end])
            _res.extend(tmp)

            start = start + end

            count += 1
            if count > 100: # escape infinite loop
                break

        for _i, _token in enumerate(_res):
            pred.append(_token)
        PRED.append(pred)

        if _ing in pred:
            answer_cnt += 1

        SCORE.append(answer_cnt)
        
        # temp[i] = pred[0]

        if pred[0] not in temp:
            temp[i] = pred[0]

        else:
            for k, v in enumerate(pred[1:]):
                if v not in temp:
                    temp[i] = v
                    break

        # if (idx == 3) & (i == 8): # for verification
        #     import IPython; IPython.embed(colors="Linux"); exit(1)

        # additionally
        pred_v2.append(temp[i])

    MODEL_INPUT_V2.append(f"{' '.join(pred_v2)} [SEP] {list_tag} [SEP] {list_title}") 
    
    # additionally
    ANSWER_V2.append(answer_v2)
    PRED_V2.append(pred_v2)


In [28]:
df2 = pd.DataFrame()

# df['IDX'] = IDX
# df['SUB_IDX'] = SUB_IDX
# df['ORIGINAL'] = SEN_ORIGINAL
# df['MODEL_INPUT'] = MODEL_INPUT
# df['ANSWER'] = ANSWER
# df['PRED'] = PRED
# df['SCORE'] = SCORE

df2['IDX'] = IDX_V2
df2['ORIGINAL'] = SEN_ORIGINAL_V2
df2['INPUT'] = MODEL_INPUT_V2
df2['TITLE'] = TITLE
df2['ANSWER_LIST'] = ANSWER_V2
df2['PRED_LIST_TEMP'] = PRED_V2

In [29]:
cnt = 0
for i, v in enumerate(zip(df2['ORIGINAL'], df2['INPUT'], df2['TITLE'], df2['ANSWER_LIST'], df2['PRED_LIST_TEMP'])):
    print(v[0])
    print(v[1])
    print(v[2])
    print(v[3])
    print(v[4])

    cnt += 1
    print('')

    if cnt > 3: break

Fontina_cheese fresh_basil_leaves elbow_macaroni garlic_cloves pine_nuts parmesan_cheese eggs butter[SEP]60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go[SEP]Pesto Shrimp Mac and Cheese
alfredo_sauce parmesan_cheese milk butter salt [MASK] flour pepper [SEP] 60-minutes-or-less occasion main-dish eggs-dairy pasta seafood potluck shrimp cheese comfort-food pasta-rice-and-grains elbow-macaroni shellfish taste-mood to-go [SEP] Pesto Shrimp Mac and Cheese
Pesto Shrimp Mac and Cheese
['Fontina_cheese', 'fresh_basil_leaves', 'elbow_macaroni', 'garlic_cloves', 'pine_nuts', 'parmesan_cheese', 'eggs', 'butter']
['alfredo_sauce', 'parmesan_cheese', 'milk', 'butter', 'salt', '[MASK]', 'flour', 'pepper']

yellow_bell_peppers low_sodium_soy_sauce broccoli_florets red_onions water_chestnuts rice_vinegar snow_peas shrimp mushrooms garlic[SEP]weeknight 60-minutes-or-less cuisine occasion sea

# (3-2 결과)

In [31]:
file_path32 = f'{PROJECT_PATH}/tester/test_output/10-next_token_predict_2023_1030_all.tsv'

ss1 = pd.read_csv(file_path32, sep='\t')

In [32]:
ss1.head()

,IDX,ORIGINAL,INPUT,TITLE,ANSWER_LIST,PRED_LIST_TEMP
0,1,Fontina_cheese fresh_basil_leaves elbow_macaro...,alfredo_sauce parmesan_cheese milk butter salt...,Pesto Shrimp Mac and Cheese,"['Fontina_cheese', 'fresh_basil_leaves', 'elbo...","['alfredo_sauce', 'parmesan_cheese', 'milk', '..."
1,2,yellow_bell_peppers low_sodium_soy_sauce brocc...,vegetable_oil garlic_cloves water salt sugar b...,Shrimp Stir Fry,"['yellow_bell_peppers', 'low_sodium_soy_sauce'...","['vegetable_oil', 'garlic_cloves', 'water', 's..."
2,3,pork_chops cooking_oil apples brown_sugar cinn...,apple_pie_filling pork_chops water butter salt...,Pork Chops and Apples,"['pork_chops', 'cooking_oil', 'apples', 'brown...","['apple_pie_filling', 'pork_chops', 'water', '..."
3,4,havarti_cheese boneless_chicken_breasts roma_t...,romaine_lettuce chicken_breasts parmesan_chees...,Chopped Salad,"['havarti_cheese', 'boneless_chicken_breasts',...","['romaine_lettuce', 'chicken_breasts', 'parmes..."
4,5,port_wine blackberries beef_broth shallot hone...,blackberries sugar butter salt water eggs flou...,Blackberry port wine sauce,"['port_wine', 'blackberries', 'beef_broth', 's...","['blackberries', 'sugar', 'butter', 'salt', 'w..."


In [33]:
ss1.iloc[0]

IDX                                                               1
ORIGINAL          Fontina_cheese fresh_basil_leaves elbow_macaro...
INPUT             alfredo_sauce parmesan_cheese milk butter salt...
TITLE                                   Pesto Shrimp Mac and Cheese
ANSWER_LIST       ['Fontina_cheese', 'fresh_basil_leaves', 'elbo...
PRED_LIST_TEMP    ['alfredo_sauce', 'parmesan_cheese', 'milk', '...
Name: 0, dtype: object